In [2]:
library("ggpubr")
library("see")
library("tidyverse")
library(parallel)
library(lmerTest)
library(lme4)
suppressPackageStartupMessages(library(performance))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(pheatmap))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(plyr))
suppressPackageStartupMessages(library(ggrepel))
library(RColorBrewer)
library(viridis)

Loading required package: ggplot2

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ lubridate 1.9.2     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lme4

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step


Loading required package: viridisLite



In [3]:
Olnik<-read.csv('Olink.csv')

In [4]:
meta<-read.csv('/home/jupyter/BRI_Analysis/scRNA/hise_meta_data_2024-01-09.csv')
meta<-meta %>% filter(sample.visitName=='Flu Year 1 Day 0')

In [5]:
CMV=read.csv("/home//jupyter/BRI_Analysis/scRNA/CMV_20220718.csv") 
meta<-left_join(meta,CMV,by = c("subject.subjectGuid"))

In [6]:
rownames(meta)<-meta$sample.sampleKitGuid

In [7]:
Olnik<-Olnik %>% filter(sample.sampleKitGuid %in% unique(meta$sample.sampleKitGuid))

In [8]:
Olnik<-left_join(Olnik,meta, by = c("sample.sampleKitGuid"))

In [9]:
n_distinct(Olnik$sample.sampleKitGuid)

[1] 92

In [10]:
Olnik_BR1<-Olnik %>% filter(cohort.cohortGuid=='BR1')
Olnik_BR2<-Olnik %>% filter(cohort.cohortGuid=='BR2')

In [11]:
res_list_BR1<-mclapply(unique(Olnik_BR1$Assay),function(x){
Olnik_sub<-Olnik_BR1  %>% filter(sample.visitName=='Flu Year 1 Day 0',Assay==x)
res<-lm(NPX_bridged ~ CMV.IgG.Serology.Result.Interpretation, data=Olnik_sub)
res<-as.data.frame(summary(res)$coefficients)[2,]
res$factor<-c('CMV')  
res$Assay<-x
res$diff<-Olnik_sub %>%
  dplyr::group_by(Assay, CMV.IgG.Serology.Result.Interpretation) %>%
  dplyr::summarise(Mean_Expression = mean(NPX_bridged, na.rm = TRUE)) %>%
  spread(CMV.IgG.Serology.Result.Interpretation, Mean_Expression) %>%
  mutate(diff = Positive- Negative)%>% select(diff) %>% pull()
return(res)
},mc.cores=60)

res_list_BR2<-mclapply(unique(Olnik_BR2$Assay),function(x){
Olnik_sub<-Olnik_BR2  %>% filter(sample.visitName=='Flu Year 1 Day 0',Assay==x)
res<-lm(NPX_bridged ~ CMV.IgG.Serology.Result.Interpretation, data=Olnik_sub)
res<-as.data.frame(summary(res)$coefficients)[2,]
res$factor<-c('CMV')  
res$Assay<-x
res$diff<-Olnik_sub %>%
  dplyr::group_by(Assay, CMV.IgG.Serology.Result.Interpretation) %>%
  dplyr::summarise(Mean_Expression = mean(NPX_bridged, na.rm = TRUE)) %>%
  spread(CMV.IgG.Serology.Result.Interpretation, Mean_Expression) %>%
  mutate(diff = Positive- Negative)%>% select(diff) %>% pull()
return(res)
},mc.cores=60)

In [12]:
res_all_BR1<-do.call(rbind,res_list_BR1 )
res_all_BR1$padj<-p.adjust(res_all_BR1$`Pr(>|t|)`)
res_all_BR2<-do.call(rbind,res_list_BR2)
res_all_BR2$padj<-p.adjust(res_all_BR2$`Pr(>|t|)`)

In [13]:
res_all_BR1 %>%  arrange(`Pr(>|t|)`) %>% head(20)

,Estimate,Std. Error,t value,Pr(>|t|),factor,Assay,diff,padj
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
1,-0.4306915,0.12895773,-3.339788,0.001692157,CMV,PRSS27,-0.4306915,1
2,-1.1731312,0.37549376,-3.124236,0.003116312,CMV,TDGF1,-1.1731312,1
3,-0.3548906,0.11786948,-3.010878,0.004261058,CMV,NCR1,-0.3548906,1
4,-0.3572559,0.12072155,-2.959338,0.004902686,CMV,CA12,-0.3572559,1
5,0.4988565,0.17590576,2.835931,0.006823815,CMV,NOS3,0.4988565,1
6,-0.2595283,0.09534257,-2.722061,0.009195679,CMV,FAM3B,-0.2595283,1
7,-0.2795435,0.10490475,-2.664736,0.010658617,CMV,CX3CL1,-0.2795435,1
8,-0.3197163,0.12034026,-2.656769,0.010878087,CMV,CA9,-0.3197163,1
9,1.7559612,0.68364793,2.568517,0.013602219,CMV,FOLR3,1.7559612,1


In [14]:
res_all_BR2 %>% arrange(`Pr(>|t|)`) %>% head(50)

,Estimate,Std. Error,t value,Pr(>|t|),factor,Assay,diff,padj
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
1,0.23543244,0.06686381,3.521074,0.001031317,CMV,AMIGO2,0.23543244,1
2,0.74314702,0.22524444,3.299291,0.001952940,CMV,FCRL6,0.74314702,1
3,-0.76610804,0.23257629,-3.294007,0.001982413,CMV,OBP2B,-0.76610804,1
4,0.16582500,0.05203496,3.186800,0.002679839,CMV,ITGAV,0.16582500,1
5,-0.47922500,0.11242489,-4.262624,0.003734484,CMV,WISP2,-0.47922500,1
6,0.48419345,0.16147689,2.998531,0.004496167,CMV,KLRD1,0.48419345,1
7,0.44393958,0.15020978,2.955464,0.005049879,CMV,GNLY,0.44393958,1
8,-0.44433720,0.15393909,-2.886448,0.006071914,CMV,KRT5,-0.44433720,1
9,-0.51018571,0.17821131,-2.862813,0.006464150,CMV,ARSA,-0.51018571,1


In [ ]:
res_all_BR2